In [76]:
# General libraries
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, RegressorMixin
from pytorch_tabnet.tab_model import TabNetRegressor as TN
from pytorch_tabnet.augmentations import RegressionSMOTE
from sklearn.model_selection import RandomizedSearchCV

# Pytorch
import torch

# Utils functions
from utils.utils import read_datasets, kfold

In [27]:
x_train, x_test, y_train, y_test = read_datasets(
    'x_train.csv',
    'x_test.csv',
    'y_train.csv',
    'y_test.csv'
)


# convert data to tensor
x_train_tf = torch.tensor(x_train.values.astype(np.float64)) 
y_train_tf = torch.tensor(y_train.values)

x_test_tf = torch.tensor(x_test.values.astype(np.float64)) 
y_test_tf = torch.tensor(y_test.values)

In [54]:
folds = kfold()

In [144]:
class TabNetRegressor(BaseEstimator, RegressorMixin):

    def __init__(self, params={}):
        super(BaseEstimator, self).__init__()
        super(RegressorMixin, self).__init__()
        self.tabnet = TN()
        self.params = params

    def fit(self, X, y=None, params={}):
        self.tabnet.fit(X, y, **self.params)
    
    def predict(self, X, y=None):
        preds = self.tabnet.predict(X)
        return preds

    # def get_params(self, deep=True):
    #     print(123)
    #     return self._get_params('transformer_list', deep=deep)

In [122]:
model = TabNetRegressor(hyper_params)

In [123]:
model.tabnet

TabNetRegressor(n_d=8, n_a=8, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=None, output_dim=None, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)

In [125]:
max_epochs = 50

aug = RegressionSMOTE(p=0.2)

hyper_params = {
    # 'eval_name':['train'],
    'eval_metric':['rmse'],
    'max_epochs':[max_epochs],
    'patience':[50],
    'batch_size':[1024],
    # 'virtual_batch_size':[128],
    'num_workers':[0],
    'drop_last':[False],
    'augmentations':[aug],
}

# model.fit(X=x_train_tf.numpy(), y=y_train_tf.numpy())

In [126]:
preds = model.predict(x_test.to_numpy())

AttributeError: 'TabNetRegressor' object has no attribute 'network'

In [127]:
print(preds)

[[53.82802 ]
 [53.496307]
 [55.9084  ]
 [50.041664]
 [40.07789 ]
 [52.23827 ]
 [46.43348 ]
 [61.424873]
 [45.327168]
 [57.817223]
 [44.251415]
 [48.85537 ]
 [36.38262 ]
 [48.97306 ]
 [36.981903]
 [56.818787]
 [37.260918]
 [42.572914]
 [37.212006]
 [42.936867]
 [53.85042 ]
 [49.735   ]
 [37.696365]
 [38.124634]
 [45.76337 ]
 [38.430305]
 [53.168285]
 [53.394188]
 [36.229908]
 [51.050213]
 [55.097397]
 [44.851288]
 [49.26893 ]
 [47.664692]
 [56.182594]
 [48.43495 ]
 [44.449383]
 [36.393852]
 [52.454006]
 [35.73035 ]
 [50.542744]
 [40.15754 ]
 [37.173203]
 [44.504326]
 [55.765755]
 [42.008064]
 [35.920116]
 [44.30905 ]
 [51.65581 ]
 [68.3266  ]
 [53.71869 ]
 [40.022842]
 [50.84727 ]
 [60.645016]
 [38.135933]
 [44.409378]
 [43.42891 ]
 [42.292557]
 [51.394993]
 [39.030113]
 [52.55275 ]
 [45.283318]
 [57.387424]
 [37.204353]
 [55.540066]
 [47.670734]
 [46.84304 ]
 [40.56919 ]
 [50.26155 ]
 [46.074905]
 [44.048515]
 [68.09184 ]
 [54.257854]
 [38.696667]
 [44.28319 ]
 [45.704704]
 [49.469616]

In [139]:
# Call GridSearchCV()
model_cv = RandomizedSearchCV(
    estimator = TabNetRegressor(),
    param_distributions = hyper_params,
    n_iter=200,
    scoring= 'r2',
    cv = folds,
    verbose = 2,
    return_train_score=True,
    n_jobs = -1,
    refit = True
)

# Fit the model
best_model = model_cv.fit(X=x_train_tf.numpy(), y=y_train_tf.numpy()) 

# print(model_cv.best_params_)

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 1 is smaller than n_iter=200. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


ValueError: Invalid parameter patience for estimator TabNetRegressor(). Check the list of available parameters with `estimator.get_params().keys()`.

In [145]:
model.get_params()

{'params': {'eval_metric': ['rmse'],
  'max_epochs': [50],
  'patience': [50],
  'batch_size': [1024],
  'num_workers': [0],
  'drop_last': [False],
  'augmentations': [<pytorch_tabnet.augmentations.RegressionSMOTE at 0x1201f5e10>]}}